<a href="https://colab.research.google.com/github/kullawattana/thesis_2020_spacy_colab/blob/master/11_find_reference_neuralcoref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==2.1.0

In [ ]:
!pip install neuralcoref --no-binary neuralcoref

     |████████████████████████████████| 378kB 9.1MB/s 
     |████████████████████████████████| 133kB 13.8MB/s 
     |████████████████████████████████| 6.8MB 13.5MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
ERROR: botocore 1.19.18 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
    Running setup.py install for neuralcoref ... done


In [ ]:
!python3 -m spacy download en

     |████████████████████████████████| 11.1MB 794kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp36-none-any.whl size=11074435 sha256=5beeb6c69e4ebf508d37911919b9d09087b182720538547fe80ea32516c995f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-mvc2rcqa/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy
import neuralcoref

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

examples = [
    u'A small company manufactures customized bicycles.',
    u'Whenever the sales department receives an order, a new process instance is created.',
    u'A member of the sales department can then reject or accept the order for a customized bike.',
    u'In the former case, the process instance is finished.',
    u'In the latter case, the storehouse and the engineering department are informed.',
    u'The storehouse immediately processes the part list of the order and checks the required quantity of each part.',
    u'If the part is available in-house, it is reserved.',
    u'If it is not available, it is back-ordered.',
    u'In the meantime, the engineering department prepares everything for the assembling of the ordered bicycle.',
    u'If the storehouse has successfully reserved or back-ordered every item of the part list and the preparation activity has finished, the engineering department assembles the bicycle.',
    u'Afterwards, the sales department ships the bicycle to the customer and finishes the process instance.',
]

def printMentions(doc):
    for cluster in doc._.coref_clusters:
        print (cluster.mentions)

def printPronounReferences(doc):
    for token in doc:
        if token.pos_ == 'PRON' and token._.in_coref:
            for cluster in token._.coref_clusters:
                print (token.text + " => " + cluster.main.text)

def processDoc(text):
    doc = nlp(text)
    if doc._.coref_clusters:
        printMentions(doc)
        printPronounReferences(doc)

print(processDoc(examples[6]))

[the part is available in-house, it]
it => the part is available in-house
None


In [ ]:
import spacy
import neuralcoref

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)
doc1 = nlp('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

doc2 = nlp('Angela lives in Boston. She is quite happy in that city.')
for ent in doc2.ents:
    print(ent._.coref_cluster)

[My sister: [My sister, She], a dog: [a dog, him]]
Angela: [Angela, She]
Boston: [Boston, that city]


In [ ]:
import spacy
import pandas as pd
nlp = spacy.load('en_core_web_sm')
import re
#from nltk_test.tokenize import sent_tokenize   # Add nepural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp,greedyness=0.5,max_dist=50,blacklist=False)

'''
text = "They wore high boots, with their trousers tucked into them, " \
         "and had long black hair and heavy black moustaches."
'''

'''
text = str("The storehouse immediately processes the part list of the order and checks the required quantity of each part. "
+ " If the part is available in-house, it is reserved. "
+ " If it is not available, it is back-ordered. "
+ " This procedure is repeated for each item on the part list.")
'''

text = str("The storehouse immediately processes the part list of the order and checks the required quantity of each part.")
doc = nlp(text)

#การหาความสัมพันธ์ของ Pronoun โดยสร้าง Cluster ของ Coref
clusters = doc._.coref_clusters
resolved_coref = doc._.coref_resolved

#DisplaCy
token_data = []
for token in doc:
  token_row_data = [token.text, token.pos_, token.tag_, token.dep_, (token.head.text, token.head.i), 
                    token.head.pos_, [child.i for child in token.children]]
  token_data.append(token_row_data)
token_data_df = pd.DataFrame(token_data, columns=['text','POS','TAG','dependency','head','head_POS','children'])
print (token_data_df)

#การหาความสัมพันธ์ของ Pronoun
doc = nlp(text)
for token in doc:
    if token.pos_ == 'NOUN' and token._.in_coref:
        for cluster in token._.coref_clusters:
            print (token.text + " => " + cluster.main.text)

           text    POS  TAG  ...             head head_POS               children
0           The    DET   DT  ...  (storehouse, 1)     NOUN                     []
1    storehouse   NOUN   NN  ...   (processes, 3)     VERB                    [0]
2   immediately    ADV   RB  ...   (processes, 3)     VERB                     []
3     processes   VERB  VBZ  ...   (processes, 3)     VERB  [1, 2, 6, 10, 11, 18]
4           the    DET   DT  ...        (list, 6)     NOUN                     []
5          part   NOUN   NN  ...        (list, 6)     NOUN                     []
6          list   NOUN   NN  ...   (processes, 3)     VERB              [4, 5, 7]
7            of    ADP   IN  ...        (list, 6)     NOUN                    [9]
8           the    DET   DT  ...       (order, 9)     NOUN                     []
9         order   NOUN   NN  ...          (of, 7)      ADP                    [8]
10          and  CCONJ   CC  ...   (processes, 3)     VERB                     []
11       checks 

In [ ]:
import spacy
import neuralcoref
nlp =  spacy.load('en_core_web_sm') # create blank Language class
coref = neuralcoref.NeuralCoref(nlp.vocab, conv_dict={'Admin': ['man']})
nlp.add_pipe(coref, name='neuralcoref')
nlp.get_pipe('neuralcoref').set_conv_dict({'Admin': ['man']})
#doc=nlp("Doctor updated teachers. He modifies locations. Order is also modified by him. Admin creates report. 
# He also updates report. The Customer steals money. He stole items as well.")

text = "Whenever a company makes the decision to go public, its first task is to select the underwriters." \
"Underwriters act as financial midwives to a new issue. Usually they play a triple role:" \
"First they provide the company with procedural and financial advice, then they buy the issue, and finally they resell it to the public." \
"Established underwriters are careful of their reputation and will not handle a new issue unless they believe the facts have been presented fairly." \
"Thus, in addition to handling the sale of a companys issue, the underwriters in effect give their seal of approval to it." \
"They prepare a registration statement for the approval of the Securities and Exchange Commission (SEC)." \
"In addition to registering the issue with the SEC, they need to check that the issue complies with the so-called blue-sky laws of each state that regulate sales of securities within the state." \
"While the registration statement is awaiting approval, underwriters begin to firm up the issue price." \
"They arrange a road show to talk to potential investors. Immediately after they receive clearance from the SEC, underwriters fix the issue price." \
"After that they enter into a firm commitment to buy the stock and then offer it to the public, when they haven’t still found any reason not to do it."

doc=nlp(text)
resolved = doc._.coref_resolved
print(doc._.has_coref)
print(doc._.coref_clusters)
print('resolve---', resolved)
print('coref_clusters---', doc._.coref_clusters[0].mentions)

#https://github.com/huggingface/neuralcoref/issues/181

True
[a company: [a company, its], the underwriters: [the underwriters, the underwriters, their, They, they], Underwriters: [Underwriters, they, they, they, they], a new issue: [a new issue, the issue, it], Established underwriters: [Established underwriters, their, they], their seal of approval: [their seal of approval, it], a registration statement for the approval of the Securities and Exchange Commission: [a registration statement for the approval of the Securities and Exchange Commission, the registration statement], the SEC: [the SEC, the SEC], underwriters: [underwriters, They, they], the issue price: [the issue price, the issue price], underwriters: [underwriters, they, they], the stock: [the stock, it, it]]
resolve--- Whenever a company makes the decision to go public, a company first task is to select the underwriters.Underwriters act as financial midwives to a new issue. Usually Underwriters play a triple role:First Underwriters provide the company with procedural and financ